In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from loaders import *
import yaml
from yaml import load

In [10]:
def fitness(dataflow, workload, pe_dims, visited):
    '''
    Calculates the fitness of a given permutation `dataflow`. Returns the
    inverse EDP after running the timeloop mapper on this dataflow.
    '''
    data = evaluate(dataflow, workload, pe_dims)
    energy, latency = data
    inverse_EDP = 1 / (energy * latency)
    print(f"{dataflow} has a fitness of {inverse_EDP}")

    # update visited
    visited[''.join(dataflow)] = inverse_EDP
    # self.mapper_calls.append([''.join(dataflow), inverse_EDP])
    return inverse_EDP


def evaluate(dataflow, workload, pe_dims):
    '''
    Evaluates the given dataflow on -- architecture

    dataflow: computation ordering in list format
    workload: the file path to the workload this is being evaluated on
    returns tuple of energy, latency
    '''
    constraints='designs/system/constraints.yaml'
    
    # create a new constraints file with the new PE permutation
    stream = open(constraints, 'r')
    dictionary = yaml.safe_load(stream)
    # print(dictionary['constraints']['targets'][4])
    idx = 4 # PE
    dictionary['constraints']['targets'][idx]['permutation'] = dataflow

    filename = ''.join(dataflow)
    with open(f'iters/configs/{filename}.yaml', 'w') as file:
        yaml.dump(dictionary, file, default_flow_style=False)

    constraints = f'iters/configs/{filename}.yaml'
            

    sys_1x16_result = run_timeloop_mapper( # TODO: this should be run_timeloop_mapper not run_timeloop_model!
        # config,
        pe_dims,
        architecture='designs/system/arch.yaml',
        mapper='designs/_include/mapper.yaml',
        problem=workload,
        constraints=constraints 
    )

    # keeping track of mapper call count for data purposes
    # self.mapper_call_count += 1
    
    stats = open('./output_dir/timeloop-mapper.stats.txt', 'r').read()
    mapping = sys_1x16_result.mapping

    lines = stats.split('\n')
    energy = float([l for l in lines if 'Energy:' in l][0].split(' ', 2)[1])
    cycles = int([l for l in lines if 'Cycles:' in l][0].split(' ', 1)[1])
    # min_energy = min(min_energy, energy)

    print(energy, cycles)
    return energy, cycles

In [11]:
dataflow = ['R', 'S', 'P', 'Q', 'C', 'M', 'N']
workload='layer_shapes/fc1.yaml'
pe_dims={'pe_meshX': 2, 'pe_meshY': 8} 

stream = open('fc1_2x8.yaml', 'r')
visited = yaml.safe_load(stream)
results = []

population = [random.sample(dataflow, len(dataflow)) for _ in range(121)]
for candidate in population:
    print(candidate)
    permutation = ''.join(candidate)
    if permutation in visited:
        print(f'already visited {permutation}!')
        results.append([permutation, visited[permutation]])
    else:
        results.append([permutation, fitness(candidate, workload, pe_dims, visited)])

print(results)

['S', 'N', 'P', 'Q', 'R', 'M', 'C']
[INFO] 2025-05-02 05:04:47,522 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.03 627200
['S', 'N', 'P', 'Q', 'R', 'M', 'C'] has a fitness of 2.8973064784699996e-08
['N', 'Q', 'S', 'P', 'M', 'R', 'C']
[INFO] 2025-05-02 05:05:41,487 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


62.01 313600
['N', 'Q', 'S', 'P', 'M', 'R', 'C'] has a fitness of 5.1423568943784577e-08
['P', 'M', 'Q', 'R', 'S', 'C', 'N']
[INFO] 2025-05-02 05:06:05,520 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


...

52.63 313600
['M', 'C', 'Q', 'P', 'S', 'N', 'R'] has a fitness of 6.058855235044807e-08
['C', 'N', 'M', 'Q', 'P', 'S', 'R']
already visited CNMQPSR!
['C', 'Q', 'S', 'P', 'R', 'N', 'M']
[INFO] 2025-05-02 05:12:48,542 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


55.18 313600
['C', 'Q', 'S', 'P', 'R', 'N', 'M'] has a fitness of 5.7788610188548055e-08
['P', 'S', 'C', 'N', 'M', 'R', 'Q']
[INFO] 2025-05-02 05:13:02,523 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


RuntimeError: 

========================================================================================================================
Timeloop mapper failed with return code 0. Please check the output files in ./output_dir for more information. To debug, you can edit the file:
	./output_dir/parsed-processed-input.yaml
and run 
	tl mapper ./output_dir/parsed-processed-input.yaml
to see the error. If you're running the mapper and Timeloop can't find a vaild mapping, try setting 'diagnostics: true' in the mapper input specification.

In [ ]:
# saving this shit so we can save some time later
with open('fc1_2x8.yaml', 'w') as file:
            yaml.dump(visited, file, default_flow_style=False)